In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset, Audio
import jiwer
from IPython.display import Audio as IPyAudio, display
import librosa
import librosa.display

In [ ]:
# Load Dataset in Streaming Mode
dataset = load_dataset(
    "openslr/librispeech_asr", 
    "clean", 
    split="test", 
    streaming=True, 
    trust_remote_code=True  
)

# Manually collect a subset of 5 samples for testing.
subset = []
for i, example in enumerate(dataset):
    if i >= 5:
        break
    subset.append(example)


In [ ]:
# Data Exploration – Audio Playback & Visualization
sample = subset[0]
audio_data = sample["audio"]["array"]
sampling_rate = sample["audio"]["sampling_rate"]

display(IPyAudio(data=audio_data, rate=sampling_rate))

# Plot the audio waveform.
time_axis = np.linspace(0, len(audio_data) / sampling_rate, num=len(audio_data))
plt.figure(figsize=(14, 4))
plt.plot(time_axis, audio_data)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Clean Audio Waveform")
plt.show()

# Plot a spectrogram for additional insight.
plt.figure(figsize=(14, 4))
S = librosa.stft(audio_data)
S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
librosa.display.specshow(S_db, sr=sampling_rate, x_axis='time', y_axis='hz')
plt.colorbar(format='%+2.0f dB')
plt.title("Clean Audio Spectrogram")
plt.show()


In [ ]:
# Define Noise Injection Functions

def apply_gaussian_noise(audio_array, intensity=1.0):
    """Add Gaussian noise scaled by intensity."""
    noise = np.random.normal(0, 0.02 * intensity, size=audio_array.shape)
    return audio_array + noise

def apply_realistic_noise(audio_array, intensity=1.0):
    """
    Placeholder for realistic noise.
    For now, we simulate it as a different Gaussian noise (could later be replaced by your fuzzer).
    You might eventually add effects like pitch scaling or reverb.
    """
    noise = np.random.normal(0, 0.03 * intensity, size=audio_array.shape)
    return audio_array + noise

def apply_noise(audio_array, noise_type="clean", intensity=1.0):
    """
    Apply noise based on noise_type:
      - "clean": No noise is added.
      - "gaussian": Add Gaussian noise.
      - "realistic": Add realistic (fuzzed) noise.
    """
    if noise_type == "clean":
        return audio_array
    elif noise_type == "gaussian":
        return apply_gaussian_noise(audio_array, intensity)
    elif noise_type == "realistic":
        return apply_realistic_noise(audio_array, intensity)
    else:
        raise ValueError("Unknown noise type")


In [ ]:
# Define Metric Functions

def compute_snr(clean, noisy):
    """Compute Signal-to-Noise Ratio (SNR) in dB."""
    noise = clean - noisy
    snr = 10 * np.log10(np.sum(clean**2) / np.sum(noise**2) + 1e-10)
    return snr

def compute_mcd(clean, noisy, sr=16000, n_mfcc=13):
    """
    Compute Mean Cepstral Distortion (MCD) between clean and noisy signals.
    This is a rough approximation using MFCCs.
    """
    mfcc_clean = librosa.feature.mfcc(y=clean, sr=sr, n_mfcc=n_mfcc)
    mfcc_noisy = librosa.feature.mfcc(y=noisy, sr=sr, n_mfcc=n_mfcc)
    diff = mfcc_clean - mfcc_noisy
    # Euclidean distance per frame, averaged
    dist = np.sqrt(np.sum(diff**2, axis=0))
    mcd = (10.0 / np.log(10)) * np.sqrt(2) * np.mean(dist)
    return mcd


In [ ]:
# Load the Model and Processor
model_name = "openai/whisper-large-v3-turbo"  # Change if desired

processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


In [ ]:
# Evaluate Over Multiple Noise Conditions

# Define the noise conditions and, optionally, a list of intensities for a sweep.
noise_conditions = ["clean", "gaussian", "realistic"]
intensity = 1.0  # For simplicity, use a single intensity value; you could loop over multiple values.

# Initialize dictionaries to store metrics per noise condition.
metrics = {noise: {"ground_truths": [], "predictions": [], "snrs": [], "mcds": []} 
           for noise in noise_conditions}

for noise in noise_conditions:
    print(f"Evaluating noise condition: {noise}")
    for example in subset:
        # Extract clean audio data.
        audio = example["audio"]
        sampling_rate = audio.get("sampling_rate", 16000)
        clean_audio = audio["array"]
        
        # Apply noise according to the condition.
        noisy_audio = apply_noise(clean_audio, noise_type=noise, intensity=intensity)
        
        # Preprocess for model inference.
        input_features = processor(noisy_audio, sampling_rate=sampling_rate, return_tensors="pt").input_features
        input_features = input_features.to(device)
        
        # Generate transcription.
        with torch.no_grad():
            predicted_ids = model.generate(input_features)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # Save ground truth and prediction.
        metrics[noise]["ground_truths"].append(example["text"])
        metrics[noise]["predictions"].append(transcription)
        
        # Compute SNR and MCD (comparing noisy audio to clean audio).
        snr_val = compute_snr(clean_audio, noisy_audio)
        mcd_val = compute_mcd(clean_audio, noisy_audio, sr=sampling_rate)
        metrics[noise]["snrs"].append(snr_val)
        metrics[noise]["mcds"].append(mcd_val)
        
        # Print for inspection.
        print(f"Noise: {noise}")
        print("Ground Truth:", example["text"])
        print("Prediction:  ", transcription)
        print(f"SNR: {snr_val:.2f} dB, MCD: {mcd_val:.2f}")
        print("-" * 30)
    print("\n")
    
# Compute aggregate metrics using jiwer for transcription quality.
results = {}
for noise in noise_conditions:
    wer_val = jiwer.wer(metrics[noise]["ground_truths"], metrics[noise]["predictions"])
    cer_val = jiwer.cer(metrics[noise]["ground_truths"], metrics[noise]["predictions"])
    avg_snr = np.mean(metrics[noise]["snrs"])
    avg_mcd = np.mean(metrics[noise]["mcds"])
    results[noise] = {"WER": wer_val, "CER": cer_val, "Avg_SNR": avg_snr, "Avg_MCD": avg_mcd}

print("Aggregate Evaluation Results:")
for noise, res in results.items():
    print(f"{noise.capitalize()} Condition:")
    print(f"  WER (word error rate): {res['WER']:.2%}")
    print(f"  CER (character error rate): {res['CER']:.2%}")
    print(f"  Average SNR (signal-to-noise ratio): {res['Avg_SNR']:.2f} dB")
    print(f"  Average MCD (mean cepstral distortion): {res['Avg_MCD']:.2f}")
    print("-" * 40)


In [ ]:
# Cell 8: Qualitative Evaluation – Visualize Clean vs. Noisy Audio

# Select one sample from the subset for a detailed qualitative comparison.
sample = subset[0]
clean_audio = sample["audio"]["array"]
sampling_rate = sample["audio"]["sampling_rate"]

# Generate noisy versions for each noise condition.
noisy_versions = {noise: apply_noise(clean_audio, noise_type=noise, intensity=1.0) for noise in noise_conditions if noise != "clean"}
noisy_versions["clean"] = clean_audio  # Include clean audio for comparison

# Display audio players for each condition.
for noise, audio_version in noisy_versions.items():
    print(f"Audio for {noise.capitalize()} condition:")
    display(IPyAudio(data=audio_version, rate=sampling_rate))

# Plot spectrograms side-by-side for visual comparison.
fig, axes = plt.subplots(1, len(noisy_versions), figsize=(16, 4))
for ax, (noise, audio_version) in zip(axes, noisy_versions.items()):
    S = librosa.stft(audio_version)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    img = librosa.display.specshow(S_db, sr=sampling_rate, x_axis='time', y_axis='hz', ax=ax)
    ax.set_title(f"{noise.capitalize()} Audio")
    ax.label_outer()
fig.colorbar(img, ax=axes, format='%+2.0f dB')
plt.show()


In [ ]:
# Visualize Metrics Summary Across Noise Conditions
import pandas as pd

# Create a DataFrame from the aggregate results.
df_results = pd.DataFrame(results).T
df_results.index.name = "Noise Condition"
print(df_results)

# Plot a bar chart for WER and CER.
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

df_results["WER"].plot(kind="bar", ax=ax[0], color="skyblue", title="WER by Noise Condition")
ax[0].set_ylabel("WER")

df_results["CER"].plot(kind="bar", ax=ax[1], color="salmon", title="CER by Noise Condition")
ax[1].set_ylabel("CER")

plt.tight_layout()
plt.show()

